In [ ]:
import matplotlib.pyplot as pp
import numpy as np
import pandas as pd
import pandas_datareader.data as datareader
import tensorflow as tf
import tensorflow_probability as tp

from collections import OrderedDict

In [ ]:
%config InlineBackend.figure_format = 'svg'

In [ ]:
pp.style.use('ggplot')

In [ ]:
prior_params = OrderedDict([
    ('AAPL', (0.05, 0.03)),
    ('AMZN', (0.03, 0.02)),
    ('GOOG', (-0.03, 0.04)),
    ('TSLA', (-0.02, 0.01)),
])

stocks = prior_params.keys()

In [ ]:
session = tf.Session()

pp.figure(figsize=(12, 6))
for i, (name, params) in enumerate(prior_params.items()):
    x = tf.linspace(start=-0.15, stop=0.15, num=100)
    pp.subplot(2, 2, i + 1)
    y = tp.distributions.Normal(loc=params[0], scale=params[1]).prob(x)
    [x_, y_] = session.run([x, y])
    pp.fill_between(x_, 0, y_, alpha=0.6)
    pp.title(name)
    pp.vlines(0, 0, y_.max(), 'k', '--')
    pp.xlim(-0.15, 0.15)
pp.tight_layout()

In [ ]:
start_date, end_date = '2016-01-01', '2018-12-01'

prices = pd.DataFrame()
for name in stocks:
    series = datareader.DataReader(name, 'iex', start_date, end_date)['close']
    prices[name] = series

prices.index = pd.to_datetime(prices.index, format='%Y-%m-%d')
returns = prices.pct_change()[1:]

In [ ]:
((returns + 1).cumprod() - 1).plot(figsize=(12, 6))

In [ ]:
prior_mu = tf.constant([x[0] for x in prior_params.values()], dtype=tf.float32)
prior_sd = tf.constant([x[1] for x in prior_params.values()], dtype=tf.float32)

observed_mu = tf.to_float(returns.mean())
observed_cov = tf.to_float(returns.cov().values)

print('Observed mean:', session.run(observed_mu), sep='\n')
print('Observed covariance:', session.run(observed_cov), sep='\n')